In [26]:
import pandas as pd

## Shot data

In [27]:
# Read in shot data
df = pd.read_pickle('shot_data.pkl')

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8884 entries, 0 to 132
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   GRID_TYPE            8884 non-null   object
 1   GAME_ID              8884 non-null   object
 2   GAME_EVENT_ID        8884 non-null   int64 
 3   PLAYER_ID            8884 non-null   int64 
 4   PLAYER_NAME          8884 non-null   object
 5   TEAM_ID              8884 non-null   int64 
 6   TEAM_NAME            8884 non-null   object
 7   PERIOD               8884 non-null   int64 
 8   MINUTES_REMAINING    8884 non-null   int64 
 9   SECONDS_REMAINING    8884 non-null   int64 
 10  EVENT_TYPE           8884 non-null   object
 11  ACTION_TYPE          8884 non-null   object
 12  SHOT_TYPE            8884 non-null   object
 13  SHOT_ZONE_BASIC      8884 non-null   object
 14  SHOT_ZONE_AREA       8884 non-null   object
 15  SHOT_ZONE_RANGE      8884 non-null   object
 16  SHOT_DI

In [29]:
df['ACTION_TYPE'].value_counts()
# Maybe use percentage of attempts that are jump shots and percentage of attempts that are layups as variables

Jump Shot                             6184
Layup Shot                             743
Driving Layup Shot                     520
Pullup Jump shot                       491
Running Jump Shot                      156
Turnaround Jump Shot                   155
Fadeaway Jump Shot                     110
Step Back Jump shot                     75
Jump Bank Shot                          71
Floating Jump shot                      55
Running Layup Shot                      43
Driving Finger Roll Layup Shot          36
Running Pull-Up Jump Shot               35
Driving Floating Jump Shot              33
Running Bank shot                       30
Reverse Layup Shot                      22
Alley Oop Layup shot                    18
Turnaround Fadeaway shot                16
Driving Reverse Layup Shot              15
Driving Jump shot                       15
Cutting Layup Shot                      12
Driving Bank shot                        8
Driving Floating Bank Jump Shot          5
Tip Shot   

### Create new variables

#### Season

In [30]:
import datetime

In [31]:
df['GAME_DATE'].head()

0    20040520
1    20040520
2    20040520
3    20040520
4    20040520
Name: GAME_DATE, dtype: object

In [32]:
df['gamedate'] = pd.to_datetime(df['GAME_DATE'])

In [33]:
df['gamedate'].head()

0   2004-05-20
1   2004-05-20
2   2004-05-20
3   2004-05-20
4   2004-05-20
Name: gamedate, dtype: datetime64[ns]

In [34]:
df['season'] = pd.DatetimeIndex(df.gamedate).year

In [35]:
df.season.head()

0    2004
1    2004
2    2004
3    2004
4    2004
Name: season, dtype: int64

In [47]:
df['team'] = 'PHO'

#### Shot Zone percentages

In [36]:
df['SHOT_ZONE_BASIC'].unique()

array(['Above the Break 3', 'Mid-Range', 'In The Paint (Non-RA)',
       'Left Corner 3', 'Right Corner 3', 'Restricted Area', 'Backcourt'],
      dtype=object)

In [37]:
df['SHOT_ZONE_AREA'].unique()

array(['Left Side Center(LC)', 'Left Side(L)', 'Right Side(R)',
       'Back Court(BC)', 'Center(C)', 'Right Side Center(RC)'],
      dtype=object)

In [38]:
df['SHOT_TYPE'].unique()

array(['3PT Field Goal', '2PT Field Goal'], dtype=object)

In [39]:
df['SHOT_ATTEMPTED_FLAG'].value_counts()

1    8884
Name: SHOT_ATTEMPTED_FLAG, dtype: int64

In [40]:
import numpy as np

In [41]:
df.dtypes

GRID_TYPE                      object
GAME_ID                        object
GAME_EVENT_ID                   int64
PLAYER_ID                       int64
PLAYER_NAME                    object
TEAM_ID                         int64
TEAM_NAME                      object
PERIOD                          int64
MINUTES_REMAINING               int64
SECONDS_REMAINING               int64
EVENT_TYPE                     object
ACTION_TYPE                    object
SHOT_TYPE                      object
SHOT_ZONE_BASIC                object
SHOT_ZONE_AREA                 object
SHOT_ZONE_RANGE                object
SHOT_DISTANCE                   int64
LOC_X                           int64
LOC_Y                           int64
SHOT_ATTEMPTED_FLAG             int64
SHOT_MADE_FLAG                  int64
GAME_DATE                      object
HTM                            object
VTM                            object
gamedate               datetime64[ns]
season                          int64
dtype: objec

In [42]:
df['topkey3'] = np.where(df['SHOT_ZONE_BASIC']=='Above the Break 3', True, False)
df['rcorner3'] = np.where(df['SHOT_ZONE_BASIC']=='Right Corner 3', True, False)
df['lcorner3'] = np.where(df['SHOT_ZONE_BASIC']=='Left Corner 3', True, False)
df['paint'] = np.where(df['SHOT_ZONE_BASIC']=='In The Paint (Non-RA)', True, False)
df['3pt'] = np.where(df['SHOT_TYPE']=='3PT Field Goal', True, False)
df['2pt'] = np.where(df['SHOT_TYPE']=='2PT Field Goal', True, False)

In [43]:
# points from field goals made
def get_points(x, y): 
    if x=='3PT Field Goal' and y==1:
        return 3
    elif x=='2PT Field Goal' and y==1:
        return 2
    else:
        return 0

df['pts'] = df[['SHOT_TYPE', 'SHOT_MADE_FLAG']].apply(lambda x: get_points(*x), axis=1)

In [44]:
df.pts.value_counts()

0    5026
2    2256
3    1602
Name: pts, dtype: int64

In [45]:
print(df.topkey3.value_counts())
print(df.rcorner3.value_counts())
print(df.lcorner3.value_counts())
print(df.paint.value_counts())
print(df['3pt'].value_counts())
print(df['2pt'].value_counts())

False    4783
True     4101
Name: topkey3, dtype: int64
False    8804
True       80
Name: rcorner3, dtype: int64
False    8714
True      170
Name: lcorner3, dtype: int64
False    7931
True      953
Name: paint, dtype: int64
False    4506
True     4378
Name: 3pt, dtype: int64
True     4506
False    4378
Name: 2pt, dtype: int64


In [46]:
df['SHOT_ZONE_BASIC'].value_counts()

Above the Break 3        4101
Mid-Range                2096
Restricted Area          1457
In The Paint (Non-RA)     953
Left Corner 3             170
Right Corner 3             80
Backcourt                  27
Name: SHOT_ZONE_BASIC, dtype: int64

In [50]:
# Aggregate columns for shot zone totals
shotzones = df.groupby('gamedate', as_index=False).agg({'topkey3':'sum', 'rcorner3':'sum', 'lcorner3':'sum',
                                                        'paint':'sum', '3pt':'sum', '2pt':'sum', 
                                                        'SHOT_ATTEMPTED_FLAG':'sum', 'pts':'sum',
                                                        'team':'first', 'HTM':'first', 'VTM':'first'})

In [51]:
shotzones.head()

,gamedate,topkey3,rcorner3,lcorner3,paint,3pt,2pt,SHOT_ATTEMPTED_FLAG,pts,team,HTM,VTM
0,2004-05-20,6,0,0,2,6,7,13,17,PHO,PHO,SAC
1,2004-05-22,5,1,1,3,7,8,15,21,PHO,CON,PHO
2,2004-05-28,5,0,0,1,5,9,14,17,PHO,PHO,SEA
3,2004-06-01,5,0,0,2,5,7,12,13,PHO,PHO,HOU
4,2004-06-03,4,0,0,1,4,6,10,6,PHO,SEA,PHO


In [52]:
shotzones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541 entries, 0 to 540
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   gamedate             541 non-null    datetime64[ns]
 1   topkey3              541 non-null    int64         
 2   rcorner3             541 non-null    int64         
 3   lcorner3             541 non-null    int64         
 4   paint                541 non-null    int64         
 5   3pt                  541 non-null    int64         
 6   2pt                  541 non-null    int64         
 7   SHOT_ATTEMPTED_FLAG  541 non-null    int64         
 8   pts                  541 non-null    int64         
 9   team                 541 non-null    object        
 10  HTM                  541 non-null    object        
 11  VTM                  541 non-null    object        
dtypes: datetime64[ns](1), int64(8), object(3)
memory usage: 50.8+ KB


In [53]:
shotzones['season'] = pd.DatetimeIndex(shotzones.gamedate).year

In [54]:
shotzones.head()

,gamedate,topkey3,rcorner3,lcorner3,paint,3pt,2pt,SHOT_ATTEMPTED_FLAG,pts,team,HTM,VTM,season
0,2004-05-20,6,0,0,2,6,7,13,17,PHO,PHO,SAC,2004
1,2004-05-22,5,1,1,3,7,8,15,21,PHO,CON,PHO,2004
2,2004-05-28,5,0,0,1,5,9,14,17,PHO,PHO,SEA,2004
3,2004-06-01,5,0,0,2,5,7,12,13,PHO,PHO,HOU,2004
4,2004-06-03,4,0,0,1,4,6,10,6,PHO,SEA,PHO,2004


In [55]:
# Calculating percentages
shotzones['p_topkey3'] = shotzones['topkey3']/shotzones['SHOT_ATTEMPTED_FLAG']

In [56]:
shotzones['p_topkey3'].head()

0    0.461538
1    0.333333
2    0.357143
3    0.416667
4    0.400000
Name: p_topkey3, dtype: float64

In [57]:
shotzones['p_rcorner3'] = shotzones.rcorner3/shotzones['SHOT_ATTEMPTED_FLAG']

In [58]:
shotzones['p_lcorner3'] = shotzones.lcorner3/shotzones['SHOT_ATTEMPTED_FLAG']
shotzones['p_paint'] = shotzones.paint/shotzones['SHOT_ATTEMPTED_FLAG']
shotzones['r_outside_inside'] = shotzones['3pt']/shotzones['2pt']

In [59]:
shotzones.head()

,gamedate,topkey3,rcorner3,lcorner3,paint,3pt,2pt,SHOT_ATTEMPTED_FLAG,pts,team,HTM,VTM,season,p_topkey3,p_rcorner3,p_lcorner3,p_paint,r_outside_inside
0,2004-05-20,6,0,0,2,6,7,13,17,PHO,PHO,SAC,2004,0.461538,0.000000,0.000000,0.153846,0.857143
1,2004-05-22,5,1,1,3,7,8,15,21,PHO,CON,PHO,2004,0.333333,0.066667,0.066667,0.200000,0.875000
2,2004-05-28,5,0,0,1,5,9,14,17,PHO,PHO,SEA,2004,0.357143,0.000000,0.000000,0.071429,0.555556
3,2004-06-01,5,0,0,2,5,7,12,13,PHO,PHO,HOU,2004,0.416667,0.000000,0.000000,0.166667,0.714286
4,2004-06-03,4,0,0,1,4,6,10,6,PHO,SEA,PHO,2004,0.400000,0.000000,0.000000,0.100000,0.666667


#### Season dummies

In [60]:
years = shotzones.season.unique().tolist()
print(years)

[2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2016, 2017, 2018, 2019, 2020, 2021]


In [61]:
# Make this a loop later
shotzones['season04'] = np.where(shotzones.season==2004, True, False)
shotzones['season05'] = np.where(shotzones.season==2005, True, False)
shotzones['season06'] = np.where(shotzones.season==2006, True, False)
shotzones['season07'] = np.where(shotzones.season==2007, True, False)
shotzones['season08'] = np.where(shotzones.season==2008, True, False)
shotzones['season09'] = np.where(shotzones.season==2009, True, False)
shotzones['season10'] = np.where(shotzones.season==2010, True, False)
shotzones['season11'] = np.where(shotzones.season==2011, True, False)
shotzones['season12'] = np.where(shotzones.season==2012, True, False)
shotzones['season13'] = np.where(shotzones.season==2013, True, False)
shotzones['season14'] = np.where(shotzones.season==2014, True, False)
shotzones['season16'] = np.where(shotzones.season==2016, True, False)
shotzones['season17'] = np.where(shotzones.season==2017, True, False)
shotzones['season18'] = np.where(shotzones.season==2018, True, False)
shotzones['season19'] = np.where(shotzones.season==2019, True, False)
shotzones['season20'] = np.where(shotzones.season==2020, True, False)
shotzones['season21'] = np.where(shotzones.season==2021, True, False)

In [62]:
shotzones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541 entries, 0 to 540
Data columns (total 35 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   gamedate             541 non-null    datetime64[ns]
 1   topkey3              541 non-null    int64         
 2   rcorner3             541 non-null    int64         
 3   lcorner3             541 non-null    int64         
 4   paint                541 non-null    int64         
 5   3pt                  541 non-null    int64         
 6   2pt                  541 non-null    int64         
 7   SHOT_ATTEMPTED_FLAG  541 non-null    int64         
 8   pts                  541 non-null    int64         
 9   team                 541 non-null    object        
 10  HTM                  541 non-null    object        
 11  VTM                  541 non-null    object        
 12  season               541 non-null    int64         
 13  p_topkey3            541 non-null  

In [63]:
# Convert booleans to integers for regression (make loop later)
shotzones.season04 = shotzones.season04.astype(int)
shotzones.season05 = shotzones.season05.astype(int)
shotzones.season06 = shotzones.season06.astype(int)
shotzones.season07 = shotzones.season07.astype(int)
shotzones.season08 = shotzones.season08.astype(int)
shotzones.season09 = shotzones.season09.astype(int)
shotzones.season10 = shotzones.season10.astype(int)
shotzones.season11 = shotzones.season11.astype(int)
shotzones.season12 = shotzones.season12.astype(int)
shotzones.season13 = shotzones.season13.astype(int)
shotzones.season14 = shotzones.season14.astype(int)
shotzones.season16 = shotzones.season16.astype(int)
shotzones.season17 = shotzones.season17.astype(int)
shotzones.season18 = shotzones.season18.astype(int)
shotzones.season19 = shotzones.season19.astype(int)
shotzones.season20 = shotzones.season20.astype(int)
shotzones.season21 = shotzones.season21.astype(int)

shotzones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541 entries, 0 to 540
Data columns (total 35 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   gamedate             541 non-null    datetime64[ns]
 1   topkey3              541 non-null    int64         
 2   rcorner3             541 non-null    int64         
 3   lcorner3             541 non-null    int64         
 4   paint                541 non-null    int64         
 5   3pt                  541 non-null    int64         
 6   2pt                  541 non-null    int64         
 7   SHOT_ATTEMPTED_FLAG  541 non-null    int64         
 8   pts                  541 non-null    int64         
 9   team                 541 non-null    object        
 10  HTM                  541 non-null    object        
 11  VTM                  541 non-null    object        
 12  season               541 non-null    int64         
 13  p_topkey3            541 non-null  

In [86]:
# Opponent column
shotzones['opponent'] = np.where(shotzones['HTM']==shotzones['team'], shotzones['VTM'], shotzones['HTM'])
shotzones.head()

,gamedate,topkey3,rcorner3,lcorner3,paint,3pt,2pt,SHOT_ATTEMPTED_FLAG,pts,team,...,season12,season13,season14,season16,season17,season18,season19,season20,season21,opponent
0,2004-05-20,6,0,0,2,6,7,13,17,PHO,...,0,0,0,0,0,0,0,0,0,SAC
1,2004-05-22,5,1,1,3,7,8,15,21,PHO,...,0,0,0,0,0,0,0,0,0,CON
2,2004-05-28,5,0,0,1,5,9,14,17,PHO,...,0,0,0,0,0,0,0,0,0,SEA
3,2004-06-01,5,0,0,2,5,7,12,13,PHO,...,0,0,0,0,0,0,0,0,0,HOU
4,2004-06-03,4,0,0,1,4,6,10,6,PHO,...,0,0,0,0,0,0,0,0,0,SEA


In [89]:
shotzones.opponent.value_counts()

SEA    66
MIN    63
LAS    62
SAN    51
IND    42
CON    37
CHI    34
NYL    33
WAS    32
ATL    23
SAC    21
TUL    17
DET    16
HOU    15
DAL    12
LVA    11
CHA     6
Name: opponent, dtype: int64

## Pickle for later

In [90]:
df.to_pickle('df_shots.pkl')
shotzones.to_pickle('df_shotzones.pkl')

## Teams data

In [72]:
teams_df = pd.read_pickle('teams_data.pkl')

In [73]:
teams_df.info

<bound method DataFrame.info of     Season Team   W   L  Rank   G    MP   FG   FGA    FG%  ...    FT%  ORB  \
0     1999  MIN  15  17     4  32  6475  748  1925  0.389  ...  0.721  298   
1     2000  MIN  15  17     6  32  6425  770  1831  0.421  ...  0.758  258   
2     2001  MIN  12  20     6  32  6475  671  1810  0.371  ...  0.769  308   
3     2002  MIN  10  22     8  32  6475  727  1775  0.410  ...  0.663  309   
4     2003  MIN  18  16     4  34  6850  875  1978  0.442  ...  0.721  361   
..     ...  ...  ..  ..   ...  ..   ...  ...   ...    ...  ...    ...  ...   
7     2004  HOU  13  21     6  34  6850  798  1933  0.413  ...  0.745  311   
8     2005  HOU  19  15     3  34  6875  860  1920  0.448  ...  0.764  276   
9     2006  HOU  18  16     3  34  6925  928  2098  0.442  ...  0.711  303   
10    2007  HOU  13  21     5  34  6800  915  2149  0.426  ...  0.768  374   
11    2008  HOU  17  17     5  34  6925  923  2209  0.418  ...  0.765  375   

    DRB   TRB  AST  STL  BLK  T

In [74]:
# Calculate winning percentage each season
teams_df['win_loss'] = teams_df['W']/(teams_df['W'] + teams_df['L'])

In [75]:
teams_df.head()

,Season,Team,W,L,Rank,G,MP,FG,FGA,FG%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,win_loss
0,1999,MIN,15,17,4,32,6475,748,1925,0.389,...,298,608,906,525,204,78,437,670,2035,0.468750
1,2000,MIN,15,17,6,32,6425,770,1831,0.421,...,258,612,870,495,237,63,497,690,2193,0.468750
2,2001,MIN,12,20,6,32,6475,671,1810,0.371,...,308,693,1001,437,210,127,507,695,2077,0.375000
3,2002,MIN,10,22,8,32,6475,727,1775,0.410,...,309,650,959,466,226,102,529,617,2003,0.312500
4,2003,MIN,18,16,4,34,6850,875,1978,0.442,...,361,718,1079,580,242,101,558,634,2380,0.529412


In [76]:
teams_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 278 entries, 0 to 11
Data columns (total 29 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Season    278 non-null    int64  
 1   Team      278 non-null    object 
 2   W         278 non-null    int64  
 3   L         278 non-null    int64  
 4   Rank      278 non-null    int64  
 5   G         278 non-null    int64  
 6   MP        278 non-null    int64  
 7   FG        278 non-null    int64  
 8   FGA       278 non-null    int64  
 9   FG%       278 non-null    float64
 10  3P        278 non-null    int64  
 11  3PA       278 non-null    int64  
 12  3P%       278 non-null    float64
 13  2P        278 non-null    int64  
 14  2PA       278 non-null    int64  
 15  2P%       278 non-null    float64
 16  FT        278 non-null    int64  
 17  FTA       278 non-null    int64  
 18  FT%       278 non-null    float64
 19  ORB       278 non-null    int64  
 20  DRB       278 non-null    int64  

In [77]:
# Shift column down to get previous year's win percentage (win_loss)
teams_df['win_prev'] = teams_df['win_loss'].shift(periods=1)
teams_df.head()

,Season,Team,W,L,Rank,G,MP,FG,FGA,FG%,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,win_loss,win_prev
0,1999,MIN,15,17,4,32,6475,748,1925,0.389,...,608,906,525,204,78,437,670,2035,0.468750,NaN
1,2000,MIN,15,17,6,32,6425,770,1831,0.421,...,612,870,495,237,63,497,690,2193,0.468750,0.46875
2,2001,MIN,12,20,6,32,6475,671,1810,0.371,...,693,1001,437,210,127,507,695,2077,0.375000,0.46875
3,2002,MIN,10,22,8,32,6475,727,1775,0.410,...,650,959,466,226,102,529,617,2003,0.312500,0.37500
4,2003,MIN,18,16,4,34,6850,875,1978,0.442,...,718,1079,580,242,101,558,634,2380,0.529412,0.31250


In [79]:
teams_df = teams_df.rename(columns={'Season':'season', 'Team':'opponent'})

In [80]:
new_teams_df = teams_df[['season', 'opponent','win_prev']]

In [81]:
new_teams_df.head()

,season,opponent,win_prev
0,1999,MIN,NaN
1,2000,MIN,0.46875
2,2001,MIN,0.46875
3,2002,MIN,0.37500
4,2003,MIN,0.31250


## Merge

In [92]:
# Read in shot zone data
shotzones_df = pd.read_pickle('df_shotzones.pkl')

In [93]:
main_df = pd.merge(shotzones_df, new_teams_df, left_on=['season', 'opponent'], right_on=['season', 'opponent'], 
                  how='left')
main_df.head()

,gamedate,topkey3,rcorner3,lcorner3,paint,3pt,2pt,SHOT_ATTEMPTED_FLAG,pts,team,...,season13,season14,season16,season17,season18,season19,season20,season21,opponent,win_prev
0,2004-05-20,6,0,0,2,6,7,13,17,PHO,...,0,0,0,0,0,0,0,0,SAC,0.558824
1,2004-05-22,5,1,1,3,7,8,15,21,PHO,...,0,0,0,0,0,0,0,0,CON,0.529412
2,2004-05-28,5,0,0,1,5,9,14,17,PHO,...,0,0,0,0,0,0,0,0,SEA,0.529412
3,2004-06-01,5,0,0,2,5,7,12,13,PHO,...,0,0,0,0,0,0,0,0,HOU,0.588235
4,2004-06-03,4,0,0,1,4,6,10,6,PHO,...,0,0,0,0,0,0,0,0,SEA,0.529412


In [94]:
main_df.to_pickle('merge_df.pkl')